# Add Atlas to Database
## Parameters
The next code block sets parameters that are used throughout the remainder of the notebook.

In [1]:
# Required: atlas data in table format (csv or tsv) to be deposited, direct path
atlas_file_name = "/global/cfs/cdirs/metatlas/projects/1_TemplateAtlases/Custom_EPC18_Annotation20190824_Template_Lignin_unlabeled_negative.csv"

# Required: name to assign to atlas, string without spaces
atlas_name = "smkosina_lignin_NEG"

# Optional: value for mz_tolerance in ppm - defaults to keep input column values or use 5.0 ppm if not specified in input table
mz_tolerance = None

# Optional: specify the polarity if it's not a column in the input atlas table - defaults to keep input column value or infers from adducts
polarity = None

# Optional: choose to sort the atlas by RT and MZ in ascending order
sort_atlas = True

# Optional: is the atlas to be deposited an internal standard atlas (i.e., with unlabeled and labeled compounds)?
istd_atlas = False

# Optional: run a check to see if the number of compounds in deposited atlas matches number in input atlas
# defaulted to False because it can take a non-trivial amount of time for larger atlases
run_retrieval_check = False

############ The rest of this block contains project independent parameters

# to use an older version of the metatlas source code, set this to a commit id,
# branch name, or tag. If None, then use the the "main" branch.
source_code_version_id = None

# Threshold for how much status information metatlas functions print in the notebook
# levels are 'DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'
log_level = "INFO"

## Initialization

In [2]:
import logging
from pathlib import Path
from IPython.display import Markdown, display
import pandas as pd

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

assert atlas_file_name is not None
assert atlas_name is not None

logger = logging.getLogger("metatlas.jupyter")

kernel_def = """{"argv":["shifter","--entrypoint","--image=ghcr.io/biorack/metatlas/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python",
                 "metadata": { "debugger": true }}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
try:
    has_root_kernel = Path("/root/.local/share/jupyter/kernels/papermill/kernel.json").is_file()
except PermissionError:
    has_root_kernel = False
if not has_root_kernel and not kernel_file_name.is_file():
    kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    logger.critical('CRITICAL: Notebook kernel has been installed. Set kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution
try:
    from metatlas.tools import notebook
except ImportError as err:
    logger.critical('CRITICAL: Set notebook kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution from err

# Check if any required variables are None
if atlas_file_name is None or atlas_name is None:
    raise SystemExit("Exiting notebook due to unset variables in cell 1.")

notebook.setup(log_level, source_code_version_id)
from metatlas.plots.dill2plots import make_atlas_from_table
from metatlas.io.metatlas_get_data_helper_fun import make_atlas_df,sort_atlas_table

2025-07-17 19:00:03 INFO     Running on git commit: b'ce7e966b78d56ee1b7dbbf19ea81011a7e3bbc94' from 2025-07-17 11:49:17 -0700 -- 11 minutes ago


2025-07-17 19:00:06 INFO     NERSC=True


## Atlas sort

In [3]:
if sort_atlas == True:
    deposit_atlas_file_name = sort_atlas_table(atlas_file_name, "rt_peak", 'mz', istd_atlas)    
else:
    deposit_atlas_file_name = atlas_file_name

2025-07-17 19:00:17 INFO     Writing sorted atlas to: /global/cfs/cdirs/metatlas/projects/1_TemplateAtlases/Custom_EPC18_Annotation20190824_Template_Lignin_unlabeled_negative_sorted.csv


## Atlas generation

In [4]:
%%time
logger.info('Reading in atlas from ' + deposit_atlas_file_name + ' and depositing to MySQL database at NERSC')

atlas = make_atlas_from_table(deposit_atlas_file_name, atlas_name, store=True, mz_tolerance=mz_tolerance, polarity=polarity)
logger.info('Making atlas df from ' + atlas.name + ' for downstream checks')
atlas_df = make_atlas_df(atlas)
display(Markdown(f"### Atlas unique_id: {atlas.unique_id}"))
display(Markdown(f"### Atlas name: {atlas.name}"))

2025-07-17 19:00:17 INFO     Reading in atlas from /global/cfs/cdirs/metatlas/projects/1_TemplateAtlases/Custom_EPC18_Annotation20190824_Template_Lignin_unlabeled_negative_sorted.csv and depositing to MySQL database at NERSC
2025-07-17 19:00:23 CRITICAL Uncaught exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/IPython/core/magics/execution.py", line 1350, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 3, in <module>
  File "/tmp/metatlas.hPewKPJYBt/metatlas/plots/dill2plots.py", line 3474, in make_atlas_from_table
    atlas = get_atlas(atlas_name, atlas_df, polarity, mz_tolerance)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/metatlas.hPewKPJYBt/metatlas/plots/dill2plots.py", line 3367, in get_atlas
    my_id = get_compound_identification(row, polarity, mz_tolerance)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/metatlas.hPewKPJYBt/metatlas/plots/dill2p

SystemExit: 128

/usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Deposit check

In [ ]:
%%time
if run_retrieval_check == True:

    import pandas as pd
    from metatlas.datastructures.utils import get_atlas

    def atlas_id_to_df(atlas_unique_id: str) -> pd.DataFrame:
        """Retrieve atlas from database using unique id and create DataFrame from compound identification data."""

        atlas = get_atlas(atlas_unique_id)

        atlas_df = make_atlas_df(atlas)

        return atlas_df
    
    # Ensure atlas can be retrieved from database
    retrieved_atlas_df = atlas_id_to_df(atlas.unique_id)
    
    # Convert input atlas to df
    input_atlas_df = pd.read_csv(deposit_atlas_file_name)
    
    # Check dataframe dims against expectations
    if input_atlas_df.shape[0] == retrieved_atlas_df.shape[0]:
        
        logger.info('Input and deposited atlas have the same number of compounds.')
    
    else:
        
        logger.info('Warning! Input and deposited atlas do not have the same number of compounds.')